# R02 Nhóm, lookahead và lookbehind

## Mục đích

Giới thiệu cách sử dụng nhóm (grouping) và các chức năng match "xung quanh" chuỗi kí tự cần tìm kiếm.


## Nhóm

Trong bài trước chúng ta đã học cách match một nội dung nào đó trong các chuỗi kí tự nhưng chưa nói đến cách để trích xuất nội dung match để sử dụng. Ví dụ, chúng ta muốn tách riêng phần họ đệm và tên từ họ tên đầy đủ. Để làm được điều đó, bạn cần sử dụng nhóm.

In [1]:
import re

s = "Nguyễn Thị Vân"
pat = r"(.*)\s(\S*)"

m = re.search(pat, s)
if m is not None:
    print(m.groups())

('Nguyễn Thị', 'Vân')


Chuỗi RegEx trên sử dụng cặp dấu ngoặc đơn `()` để bao xung quanh **nhóm** cần trích ra. Như vậy, trong chuỗi ở ví dụ trên, chúng ta có hai nhóm (họ đệm và tên riêng). Trong Python, hàm `re.match()` và `re.search()` trả về một đối tượng match (match object), bạn có thể sử dụng hàm `group()` hoặc `groups()` của đối tượng này để truy cập vào các nhóm.

Nếu đang theo tác với series trong Pandas, bạn sẽ có một công cụ mạnh hơn cho việc này, đó là hàm `.str.extract()` và `.str.extractall()`.

In [2]:
import pandas as pd

d = pd.DataFrame({
    "id": range(1, 5),
    "name": ["Nguyễn Thị Vân", "Hoàng Văn Tuấn", "Ngô Văn Minh", "Đào Thị Thu Hà"] 
})

d["name"].str.extractall(pat)

,,0,1
,match,,
0,0,Nguyễn Thị,Vân
1,0,Hoàng Văn,Tuấn
2,0,Ngô Văn,Minh
3,0,Đào Thị Thu,Hà


Pandas tự động tạo ra series hoặc data frame tùy theo số lượng nhóm. Trong trường hợp bạn có nhiều lần match trong một chuỗi kí tự, Pandas sẽ tạo ra nhiều hàng khác nhau, mỗi hàng được đánh số khác nhau trong cấp `"match"` của index.

In [3]:
d["pers_ids"] = ["A09388", "C97471", "G33113 F42555", "E42882 H64378 D65912"]

pat = r"(\d{5})"    # Chỉ lấy phần số

d["pers_ids"].str.extractall(pat)

0
  match       
0 0      09388
1 0      97471
2 0      33113
  1      42555
3 0      42882
  1      64378
  2      65912

### Đặt tên nhóm

Trong các ví dụ với Pandas, bạn có thể thấy rằng các tên cột được trả về là 0, 1, v.v.. Chúng ta có thể đặt tên gọi thay cho các con số này để sau này đỡ phải đổi tên. Cú pháp để đặt tên nhóm là `(?P<tên_nhóm>CHUỖI_REG_EX)`.

In [4]:
pat = r"(?P<hodem>.*)\s(?P<ten>\S*)"
d["name"].str.extractall(pat)

,,hodem,ten
,match,,
0,0,Nguyễn Thị,Vân
1,0,Hoàng Văn,Tuấn
2,0,Ngô Văn,Minh
3,0,Đào Thị Thu,Hà


Thư viện `re` cũng sử dụng cách thức tương tự. Bạn có thể trích xuất nội dung match của một nhóm bằng cách cung cấp tên của nhóm đó cho hàm `group()`.

In [5]:
m = re.search(pat, s)
m.group("hodem")

'Nguyễn Thị'

### Tham chiếu nhóm đã tóm bắt

Sẽ có lúc bạn cần phải tham chiếu lại một nhóm đã tìm thấy ở phía trước. Chẳng hạn, bạn muốn match tất cả những mã số có cú pháp sau: `ab-abcd`, trong đó `a` có thể là 1 hoặc 2, `b` là một chữ số bất kì nhưng `ab` phải lặp lại, và phần `cd` bất kì. Cú pháp cho tham chiếu nhóm là `\số_thứ_tự`, số thứ tự nhóm tham chiếu có thể từ 1 đến 99. Các số 0 và có 3 chữ số trở lên sẽ được coi là điều kiện tìm kiếm.

In [6]:
pat = r"([12]\d)-\1\d{2}"
ids = ["23-3211", "23-2309"]

for s in ids:
    m = re.search(pat, s)
    print(f"{s}:", m is not None)

23-3211: False
23-2309: True


Bạn cũng có thể tham chiếu tới nhóm có tên bằng cú pháp `(?P=tên)`.

In [7]:
pat = r"(?P<ab>[12]\d)-(?P=ab)\d{2}"
ids = ["23-3211", "23-2309"]

for s in ids:
    m = re.search(pat, s)
    print(f"{s}:", m is not None)

23-3211: False
23-2309: True


## Tìm xung quanh

Bạn có thể đặt điều kiện match **chỉ khi** phía sau (lookhead) hoặc phía trước (lookbehind) của một chuỗi kí tự điều kiện thỏa mãn (positive) hoặc không thỏa mãn (negative) một điều kiện nào đó. Ví dụ, tìm tất cả những mã số bắt đầu bằng 4 sau dấu gạch ngang.

Kí hiệu       | Ý nghĩa
--------------|---------------------------------------------------
`(?=)`        | Positive lookahead
`(?!)`        | Negative lookahead
`(?<=)`       | Positive lookbehind
`(?<!)`       | Negative lookbehind

In [8]:
ids = ["23-4211", "42-2309"]

pat = "4\d*"    # Nếu không có lookaround

for s in ids:
    m = re.search(pat, s)
    print(f"{s}:", m is not None)

23-4211: True
42-2309: True


In [9]:
pat = "(?<=-)4\d*"    # Nếu có lookaround

for s in ids:
    m = re.search(pat, s)
    print(f"{s}:", m is not None)

23-4211: True
42-2309: False


## Ví dụ

### Ví dụ 1

Trích xuất nội dung trong cặp dấu ngoặc kép.

In [10]:
s = "Long says: \"You can do everything with Python\", and smiles."
pat = r"(?<=\")([^\"]*)(?=\")"

m = re.search(pat, s)
print(m.group(1))

You can do everything with Python


### Ví dụ 2

Bỏ qua các dòng có dấu thăng `#` ở đầu (dòng chú thích) và trích xuất cặp key-value phân cách với nhau bởi dấu hai chấm `:`. Ví dụ:

```
color_border: 4b9213
# alpha: transparency
alpha: 0.4
```

In [11]:
lines = pd.Series([
    "color_border: #4b9213",
    "# alpha: transparency",
    "alpha: 0.4"
])

pat = r"^(?<!#)(?P<key>\w+):\s*(?P<value>.+)"
lines.str.extractall(pat)

,,key,value
,match,,
0,0,color_border,#4b9213
2,0,alpha,0.4


### Ví dụ 3

Tìm các số nguyên không âm.

In [12]:
s = "123 -100 53.2 90 -32. 444 .553 27.27.27 76"
pat = r"(?:\s|^)(\d+)(?:\s|$)"
re.findall(pat, s)

['123', '90', '444', '76']

Giải thích: `(\d+)` tìm kiếm những chuỗi kí tự số (`\d`) liên tiếp có ít nhất 1 kí tự (`+`), nhưng các chuỗi kí tự này phải đi trước và đi sau bằng dấu cách (`\s`) hoặc kí tự bắt đầu (`^`) và kết thúc (`$`) chuỗi kí tự / dòng. Như vậy chúng ta sẽ loại bỏ các trường hợp phía trước dãy số là dấu âm hoặc dấu thập phân, và các trường hợp số thập phân (có dấu thập phân ở giữa hoặc cuối số). Ở đây mình sử dụng một cú pháp chưa giới thiệu ở đầu bài là **non-capturing group** (`(?:)`). Những kí tự trong non-capturing group sẽ không được đưa vào kết quả trích xuất.

---

[Bài trước](./01_basic.ipynb) - [Danh sách bài](../README.md) - [Bài sau](./03_examples.ipynb)